#### Import Libraries

In [ ]:
# Import python packages
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import any_value, max,min,avg,call_function, month, year, dayofmonth, parse_json,split,substr,hour,replace,concat,col,lit,array_slice,array_agg,object_construct,to_time, date_add,to_date, current_date
from snowflake.snowpark.types import StringType,VariantType, DateType, IntegerType,DecimalType

import pandas as pd
import altair as alt



# Write directly to the app
st.title("Weather for Each Event in the North :sunny:")
st.write(
    """This app shows the weather that may affect Northern Trains).
    """
)

# Get the current credentials
session = get_active_session()

#### View the Events which also include Latitude and Longitude as we did previously

We are also adding a point column, this is so we can join this data with the weather data which also has points

In [ ]:
st.markdown('#### A dataframe which shows all the previously loaded events')
from snowflake.snowpark.functions import to_geography

events = session.table('BUILD_UK.DATA.EVENTS_IN_THE_NORTH').select('EVENT_DATA')
events = events.join_table_function('flatten',parse_json('EVENT_DATA'))

events = events.select(col('VALUE')['DATE'].astype(DateType()).alias('"Date"'),
                       col('VALUE')['DESCRIPTION'].astype(StringType()).alias('"Description"'),
                       col('VALUE')['NAME'].astype(StringType()).alias('"Name"'),
                       to_geography(parse_json('VALUE')['CENTROID']).alias('"Centroid"'),
                       call_function('H3_POINT_TO_CELL_STRING',col('"Centroid"'),lit(5)).alias('"H3"'),
                       call_function('H3_CELL_TO_BOUNDARY',col('"H3"')).alias('"GEOM"'),
                      col('VALUE')['COLOR'].alias('"Colour"'))
#events_latlon = events_latlon.with_column('POINT',call_function('ST_MAKEPOINT',col('"Longitude"'),col('"Latitude"')))
events

#### Bring in the Weather Data - in this case we are filtering on the most recent weather forecasts

In [ ]:

### load in daily forecast

weather_daily = session.table('POSTCODE_SECTOR_WEATHER_FORECASTS__ADVANCED_WITH_SOLAR.PCSECT_FORECAST."advanced_with_solar_daily_view"')




### find the latest weather forecast
weather_daily_max = weather_daily.agg(max('"Issued_at"').alias('MAX'))

### join the latest issue date to the dataset in order to filter it.
weather_daily = weather_daily.join(weather_daily_max,weather_daily_max['MAX']==weather_daily['"Issued_at"']).drop('MAX')
st.markdown('##### Daily forecast')
st.dataframe(weather_daily.limit(10))



In [ ]:
daily_h3 = weather_daily.with_column('H3',call_function('H3_POINT_TO_CELL_STRING',col('POINT'),5))

#### Joining relevant weather forecasts for each Event

In [ ]:
from snowflake.snowpark.functions import to_geography
eventsj = events.join(daily_h3,call_function('H3_TRY_GRID_DISTANCE',events['"H3"'], daily_h3['H3']).isNotNull(),lsuffix='L')

eventsj.limit(1)
eventsj = eventsj.group_by('"Date"','"Validity_date"','"Name"',)\
.agg(avg('"Max_temperature_day"').alias('"Temperature Day"'),
     avg('"Min_temperature_night"').alias('"Temperature Night"'),
     avg('"Max_Wind_Gust_day"').alias('"Wind Gust Day"'),
     avg('"Max_Wind_Gust_night"').alias('"Wind Gust Night"'),
avg('"Probability_of_Rain_day"').alias('"Probability of Rain Day"'),
avg('"Probability_of_Rain_night"').alias('"Probability of Rain Night"'),
avg('"Probability_of_Heavy_Rain_day"').alias('"Probability of Heavy Rain Day"'),
avg('"Probability_of_Heavy_Rain_night"').alias('"Probability of Heavy Rain Night"'),
avg('"Probability_of_Snow_night"').alias('"Probability of Snow Night"'),
avg('"Probability_of_Snow_day"').alias('"Probability of Snow Day"'),
avg('"Probability_of_Hail_night"').alias('"Probability of Hail Night"'),
avg('"Probability_of_Hail_day"').alias('"Probability of Hail Day"'),
avg('"Probability_of_Fog_night"').alias('"Probability of Fog Night"'),
avg('"Probability_of_Fog_day"').alias('"Probability of Fog Day"'),
avg('"Maximum_UV_index"').alias('"Maximum UV index"')
                                                          )

eventsj


### Create a Human Readable Weather Forecast

#### STEP 1 - Create Data Filters

In [ ]:
event_filter = events.select('"Name"')
date_filter = eventsj.agg(max('"Validity_date"').alias('MAX'),
                          min('"Validity_date"').alias('MIN')).to_pandas()




selected_event = st.selectbox('Select Event:',event_filter)
selected_date = st.date_input('Select Date:',date_filter.MIN.iloc[0],date_filter.MIN.iloc[0],date_filter.MAX.iloc[0])

#### Create the Prompt

In [ ]:
todaypd = eventsj.filter((col('"Validity_date"')==date_add(lit(selected_date),1))
                                     & (col('"Name"')==selected_event)).to_pandas()

st.write(todaypd)
melt = pd.melt(todaypd)
melt['variable'] = melt['variable'].astype("string")
melt['value'] = melt['value'].astype("string")


##### create an object to feed into the LLM
object = session.create_dataframe(melt)
object = object.with_column('object',object_construct(col('"variable"'),col('"value"')))
    
object = object.select(array_agg('OBJECT').alias('OBJECT'))



prompt = object.select(concat(lit('Generate a descriptive weather report in 500 containing paragraphs words for today which includes relevant emojis to summarise the weather based on the following dataset'),
                    col('object').astype(StringType()),
                   lit('USE APPROPIATE MARKDOWN TO ENHANCE THE PRESENTATION. NO COMMENTS')).alias('PROMPT'))

st.code(prompt.to_pandas().PROMPT.iloc[0])



#### Run the LLM

In [ ]:
complete = prompt.select(call_function('snowflake.cortex.complete',lit('mixtral-8x7b'),col('PROMPT')))
st.write(complete.collect()[0][0])

#### Using Observational data with historic events

In [ ]:
events

In [ ]:
obs = session.table('POSTCODE_SECTOR_WEATHER_FORECASTS__ADVANCED_WITH_SOLAR.PCSECT_FORECAST."daily_obs_view"')
obs = obs.with_column('H3',call_function('H3_POINT_TO_CELL_STRING',col('POINT'),5))
obs.limit(5)

In [ ]:
alter warehouse BUILD_UK_WAREHOUSE set warehouse_size = 'medium'

Below, we will join the observation data to the event data by H3.  We will also join by date - however, as we are looking at historic, we will only join to the month and day of the date so we see what the weather is like for the same date but for previous years.  As we are looking though historic data, we have increased the warehouse size to give us a bit more power.

In [ ]:
obs = obs.filter(year('"Validity_date"')>2019)

event_obs = events.join(obs, (dayofmonth( obs['"Validity_date"']) == dayofmonth(events['"Date"'])) &
                        (month( obs['"Validity_date"']) == month(events['"Date"']))&
                       (events['"H3"'] == obs['H3']),lsuffix='_obs')

cache = event_obs.cache_result()

In [ ]:
cache.count()

#### Take a look at the results below.  The query has been sorted by event name and Validity date - you can compare rain and description of the weather of previous years.

In [ ]:
cache = cache.group_by('"Date"','"Description"','"Name"', '"Validity_date"').agg(avg('"Total_rainfall_amount"'),any_value('"Predominant_weather_description"'))
cache.sort('"Name"','"Validity_date"')